In [ ]:
# Baseline Simulator

In [45]:
# necessary libraries: Circuit Construction, transpilation, For simulation, visualization, JSON
# Circuit Construction
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

#Transpilation
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# For Simulation
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
aer_sim = AerSimulator()                              # for ideal simulation
service = QiskitRuntimeService()

real_backend = service.backend("ibm_sherbrooke")
aer = AerSimulator.from_backend(real_backend)           # for noisy simulation

from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler_ideal = Sampler(backend=aer_sim)
sampler_noisy = Sampler(backend=aer)

#Visualization
from qiskit.visualization import plot_histogram


#JSON
import json


# For Fidelity
from qiskit.quantum_info.analysis import hellinger_fidelity

In [46]:
# function for Circuit Construction (GHZ state for how many number of qubits, that is num_qubit argument of the function)

def get_ghz_circ_static(num_qubit):
    if num_qubit%2==1:                                                     #odd no of qubits
        qr=QuantumRegister(num_qubit)
        cr2=ClassicalRegister(num_qubit,name="cr2")                     
        qc=QuantumCircuit(qr,cr2)
        mid_qubit_index=int((num_qubit-1)/2)
        qc.h(mid_qubit_index)
        for i in range(mid_qubit_index,num_qubit-1):
            qc.cx(i,i+1)
            qc.cx(num_qubit-i-1,num_qubit-i-2)
        

    else:                                                                    #even no of qubits
        qr=QuantumRegister(num_qubit)
        cr2=ClassicalRegister(num_qubit,name="cr2")                     
        qc=QuantumCircuit(qr,cr2)
        mid_qubit_index=int((num_qubit-2)/2)
        qc.h(mid_qubit_index)
        for i in range(mid_qubit_index,num_qubit-2):
            qc.cx(i,i+1)
            qc.cx(num_qubit-1-i-1,num_qubit-i-3)
        qc.cx(num_qubit-2,num_qubit-1)
    for i in range(num_qubit):
        qc.measure(qr[i],cr2[i])
    return qc

qc=get_ghz_circ_static(6)

In [47]:
# Transpilation ideal
pm_i = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
qc_ideal = pm_i.run(qc)

# Transpilation noisy
pm_r = generate_preset_pass_manager(backend=aer, optimization_level=3)
qc_noisy = pm_r.run(qc)

In [48]:
#Run the Circuit
result_ideal = sampler_ideal.run([qc_ideal]).result()
result_noisy = sampler_noisy.run([qc_noisy]).result()

In [51]:
# Post Processing to get Hellinger Fidelity
counts_ideal = result_ideal[0].data.cr2.get_counts()
counts_noisy = result_noisy[0].data.cr2.get_counts()
Fidelity_baseline_simulator = hellinger_fidelity(counts_ideal, counts_noisy)

In [52]:
Fidelity_baseline_simulator

0.9110448217886312

In [53]:
#Store Result into JSON file
import json
data = {
    "Fidelity_baseline_simulator" : Fidelity_baseline_simulator,    
    }


with open("Fidelity_scripts_comparison.json", "w") as f:
    json.dump(data, f, indent=4)